In [79]:
import pandas as pd
import os
import re
import openpyxl
from Bio import SeqIO

In [80]:
codontab = {
    'TCA': 'S',    # Serina
    'TCC': 'S',    # Serina
    'TCG': 'S',    # Serina
    'TCT': 'S',    # Serina
    'TTC': 'F',    # Fenilalanina
    'TTT': 'F',    # Fenilalanina
    'TTA': 'L',    # Leucina
    'TTG': 'L',    # Leucina
    'TAC': 'Y',    # Tirosina
    'TAT': 'Y',    # Tirosina
    'TAA': '*',    # Stop
    'TAG': '*',    # Stop
    'TGC': 'C',    # Cisteina
    'TGT': 'C',    # Cisteina
    'TGA': '*',    # Stop
    'TGG': 'W',    # Triptofano
    'CTA': 'L',    # Leucina
    'CTC': 'L',    # Leucina
    'CTG': 'L',    # Leucina
    'CTT': 'L',    # Leucina
    'CCA': 'P',    # Prolina
    'CCC': 'P',    # Prolina
    'CCG': 'P',    # Prolina
    'CCT': 'P',    # Prolina
    'CAC': 'H',    # Histidina
    'CAT': 'H',    # Histidina
    'CAA': 'Q',    # Glutamina
    'CAG': 'Q',    # Glutamina
    'CGA': 'R',    # Arginina
    'CGC': 'R',    # Arginina
    'CGG': 'R',    # Arginina
    'CGT': 'R',    # Arginina
    'ATA': 'I',    # Isoleucina
    'ATC': 'I',    # Isoleucina
    'ATT': 'I',    # Isoleucina
    'ATG': 'M',    # Methionina
    'ACA': 'T',    # Treonina
    'ACC': 'T',    # Treonina
    'ACG': 'T',    # Treonina
    'ACT': 'T',    # Treonina
    'AAC': 'N',    # Asparagina
    'AAT': 'N',    # Asparagina
    'AAA': 'K',    # Lisina
    'AAG': 'K',    # Lisina
    'AGC': 'S',    # Serina
    'AGT': 'S',    # Serina
    'AGA': 'R',    # Arginina
    'AGG': 'R',    # Arginina
    'GTA': 'V',    # Valina
    'GTC': 'V',    # Valina
    'GTG': 'V',    # Valina
    'GTT': 'V',    # Valina
    'GCA': 'A',    # Alanina
    'GCC': 'A',    # Alanina
    'GCG': 'A',    # Alanina
    'GCT': 'A',    # Alanina
    'GAC': 'D',    # Acido Aspartico
    'GAT': 'D',    # Acido Aspartico
    'GAA': 'E',    # Acido Glutamico
    'GAG': 'E',    # Acido Glutamico
    'GGA': 'G',    # Glicina
    'GGC': 'G',    # Glicina
    'GGG': 'G',    # Glicina
    'GGT': 'G'     # Glicina
}

In [81]:
gff = pd.read_table("ancestral.gff", comment="#", header=None,
                    names=["contig", "type", "start", "end", "strain", "description"], usecols=[0,2,3,4,6,8])
gff = gff[gff["type"] == "CDS"]
gff["ORF"] = gff["description"].apply(lambda x: re.findall(r"ID=(.+?);", x)[0])
gff["Annotation"] = gff["description"].apply(lambda x: re.findall(r";product=(.+?)$", x)[0])
gff

,contig,type,start,end,strain,description,ORF,Annotation
0,NODE_1_length_648012_cov_4.992320,CDS,447.0,1115.0,-,ID=AILJCPKN_00001;eC_number=5.4.99.23;Name=rlu...,AILJCPKN_00001,Ribosomal large subunit pseudouridine synthase D
1,NODE_1_length_648012_cov_4.992320,CDS,1598.0,2308.0,+,ID=AILJCPKN_00002;eC_number=3.2.-.-;Name=sceD_...,AILJCPKN_00002,putative transglycosylase SceD
2,NODE_1_length_648012_cov_4.992320,CDS,2948.0,3787.0,+,ID=AILJCPKN_00003;inference=ab initio predicti...,AILJCPKN_00003,hypothetical protein
3,NODE_1_length_648012_cov_4.992320,CDS,4004.0,4921.0,-,ID=AILJCPKN_00004;inference=ab initio predicti...,AILJCPKN_00004,hypothetical protein
4,NODE_1_length_648012_cov_4.992320,CDS,5289.0,6236.0,-,ID=AILJCPKN_00005;inference=ab initio predicti...,AILJCPKN_00005,hypothetical protein
...,...,...,...,...,...,...,...,...
2812,NODE_22_length_3643_cov_7.111174,CDS,2870.0,3514.0,-,ID=AILJCPKN_02813;eC_number=1.10.3.-;Name=qoxA...,AILJCPKN_02813,putative quinol oxidase subunit 2
2813,NODE_23_length_2691_cov_108.844522,CDS,916.0,1920.0,-,ID=AILJCPKN_02814;inference=ab initio predicti...,AILJCPKN_02814,hypothetical protein
2814,NODE_23_length_2691_cov_108.844522,CDS,2142.0,2627.0,+,ID=AILJCPKN_02815;Name=linA;gene=linA;inferenc...,AILJCPKN_02815,Lincosamide resistance protein
2815,NODE_24_length_2159_cov_4.289806,CDS,107.0,2095.0,-,ID=AILJCPKN_02816;eC_number=1.10.3.-;Name=qoxB...,AILJCPKN_02816,putative quinol oxidase subunit 1


In [82]:
#TROCAR AQUI O VCF
vcf = pd.read_table("perc3_filtered_calls.vcf", comment="#", header=None, usecols=[0,1,3,4],
                    names=["contig", "pos", "ref", "alt"])
def variant_type(ref, alt):
    if len(ref) > len(alt):
        return "DELETION"
    elif len(ref) < len(alt):
        return "INSERTION"
    else:
        return "SNP"
vcf["type"] = vcf.apply(lambda x: variant_type(x["ref"], x["alt"]), axis=1)
vcf

,contig,pos,ref,alt,type
0,NODE_1_length_648012_cov_4.992320,3,GCAG,GCAGGTGCAG,INSERTION
1,NODE_1_length_648012_cov_4.992320,6,GCAGGT,GCATCAGGTGCAGCAGGT,INSERTION
2,NODE_1_length_648012_cov_4.992320,115243,T,TTTTGTG,INSERTION
3,NODE_1_length_648012_cov_4.992320,423752,G,T,SNP
4,NODE_1_length_648012_cov_4.992320,485246,C,T,SNP
...,...,...,...,...,...
107,NODE_25_length_709_cov_17.411475,705,GATTT,"GATTCGAAAATCAATTT,GATTATTT",INSERTION
108,NODE_26_length_621_cov_2.697318,617,CAGGT,CAGGTGAGGT,INSERTION
109,NODE_28_length_279_cov_4.450000,1,TATT,TATTGGAGTATT,INSERTION
110,NODE_28_length_279_cov_4.450000,3,TTCA,"TTCATTGGAGTATTCA,TTTGGAGTATTCA",INSERTION


In [83]:
vcf["type"].value_counts()

type
INSERTION    55
SNP          55
DELETION      2
Name: count, dtype: int64

In [84]:
snps = vcf[["contig","type", "pos", "ref", "alt"]].values.tolist()
results = []

for index, row in gff.iterrows():
    contig_ref = row["contig"]
    start = int(row["start"])
    end = int(row["end"])
    for contig,type,snp,ref,alt in snps:
        if contig == contig_ref:
            if snp in range(start,end):
                results.append([contig,type, snp,start,end,ref,alt,row["ORF"], row["Annotation"]])
results_df = pd.DataFrame.from_records(results, columns=["Contig","Type", "SNP_POS", "Start", "End","Ref", "Alt","ORF", "Annotation"])
results_df

,Contig,Type,SNP_POS,Start,End,Ref,Alt,ORF,Annotation
0,NODE_1_length_648012_cov_4.992320,SNP,423752,423112,426795,G,T,AILJCPKN_00377,Respiratory nitrate reductase 1 alpha chain
1,NODE_1_length_648012_cov_4.992320,SNP,485246,485109,485660,C,T,AILJCPKN_00430,IS1182 family transposase ISSep1
2,NODE_1_length_648012_cov_4.992320,SNP,485486,485109,485660,G,A,AILJCPKN_00430,IS1182 family transposase ISSep1
3,NODE_1_length_648012_cov_4.992320,SNP,485534,485109,485660,T,C,AILJCPKN_00430,IS1182 family transposase ISSep1
4,NODE_1_length_648012_cov_4.992320,SNP,485569,485109,485660,A,G,AILJCPKN_00430,IS1182 family transposase ISSep1
5,NODE_4_length_369082_cov_5.973587,DELETION,341858,341444,343249,GAAAAAAAA,GAAAAAAA,AILJCPKN_01733,UDP-N-acetyl-alpha-D-glucosamine C6 dehydratase
6,NODE_5_length_222064_cov_4.235925,SNP,83145,82211,83995,C,A,AILJCPKN_01835,UvrABC system protein C
7,NODE_6_length_167623_cov_3.900032,SNP,122497,122383,122976,T,C,AILJCPKN_02105,IS1182 family transposase ISSep1
8,NODE_6_length_167623_cov_3.900032,SNP,122553,122383,122976,C,T,AILJCPKN_02105,IS1182 family transposase ISSep1
9,NODE_6_length_167623_cov_3.900032,SNP,122593,122383,122976,A,G,AILJCPKN_02105,IS1182 family transposase ISSep1


In [85]:
snps_only = results_df[results_df["Type"] == "SNP"]
for index, row in snps_only.iterrows():
    snp = row["SNP_POS"]
    start = row["Start"]
    end = row["End"]
    ref = row["Ref"]
    alt = row["Alt"]
    contig = row["Contig"]
    while start <= end:
        partial_start = start
        # print(start)
        start+=3
        partial_end = start
        if snp in range(partial_start, partial_end):
            # print("FOUND", partial_start, partial_end)
            #PARA ACHAR O SNP MANTEM O INDEX ORIGINAL -- PARA FILTRAR O PYTHON INICIA O INDEX EM ZERO...
            #ASSIM PRECISO USAR O "-1" PARA DEIXAR O INDEX CORRETO!
            for s in SeqIO.parse("ancestral.fasta", "fasta"):
                if contig == s.id:
                    codon = str(s.seq)[partial_start-1:partial_end-1]
                    to_replace = (partial_end-1) - partial_start
                    # print(f"before:{codon} -- {codontab[codon]}")
                    alt_codon = codon.replace(codon[to_replace],alt,1)
                    # print(f"after:{alt_codon} -- {codontab[alt_codon]}")
                    snps_only.loc[index, "original codon"] = codon
                    snps_only.loc[index, "original AA"] = codontab[codon]
                    snps_only.loc[index, "mutated codon"] = alt_codon
                    snps_only.loc[index, "mutated AA"] = codontab[alt_codon]
snps_only

/var/folders/hb/7z22q_f92bb8stzzy39dw7_80000gn/T/ipykernel_6679/3231343988.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_only.loc[index, "original codon"] = codon
/var/folders/hb/7z22q_f92bb8stzzy39dw7_80000gn/T/ipykernel_6679/3231343988.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_only.loc[index, "original AA"] = codontab[codon]
/var/folders/hb/7z22q_f92bb8stzzy39dw7_80000gn/T/ipykernel_6679/3231343988.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,Contig,Type,SNP_POS,Start,End,Ref,Alt,ORF,Annotation,original codon,original AA,mutated codon,mutated AA
0,NODE_1_length_648012_cov_4.992320,SNP,423752,423112,426795,G,T,AILJCPKN_00377,Respiratory nitrate reductase 1 alpha chain,AGT,S,AGT,S
1,NODE_1_length_648012_cov_4.992320,SNP,485246,485109,485660,C,T,AILJCPKN_00430,IS1182 family transposase ISSep1,TTC,F,TTT,F
2,NODE_1_length_648012_cov_4.992320,SNP,485486,485109,485660,G,A,AILJCPKN_00430,IS1182 family transposase ISSep1,AGG,R,AAG,K
3,NODE_1_length_648012_cov_4.992320,SNP,485534,485109,485660,T,C,AILJCPKN_00430,IS1182 family transposase ISSep1,TGT,C,CGT,R
4,NODE_1_length_648012_cov_4.992320,SNP,485569,485109,485660,A,G,AILJCPKN_00430,IS1182 family transposase ISSep1,CAT,H,CAG,Q
6,NODE_5_length_222064_cov_4.235925,SNP,83145,82211,83995,C,A,AILJCPKN_01835,UvrABC system protein C,CCT,P,CCA,P
7,NODE_6_length_167623_cov_3.900032,SNP,122497,122383,122976,T,C,AILJCPKN_02105,IS1182 family transposase ISSep1,TTT,F,CTT,L
8,NODE_6_length_167623_cov_3.900032,SNP,122553,122383,122976,C,T,AILJCPKN_02105,IS1182 family transposase ISSep1,GAC,D,GAT,D
9,NODE_6_length_167623_cov_3.900032,SNP,122593,122383,122976,A,G,AILJCPKN_02105,IS1182 family transposase ISSep1,AAT,N,AAG,K
10,NODE_6_length_167623_cov_3.900032,SNP,122641,122383,122976,G,A,AILJCPKN_02105,IS1182 family transposase ISSep1,GAT,D,GAA,E


In [86]:
def mutation(before,after):
    if before != after:
        return "Missense"
    elif before == after:
        return "Silence"
    elif after == "*":
        return "Nonsense"

snps_only["Mutation"] = snps_only.apply(lambda x: mutation(x["original AA"], x["mutated AA"]), axis=1)
snps_only

/var/folders/hb/7z22q_f92bb8stzzy39dw7_80000gn/T/ipykernel_6679/257270571.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_only["Mutation"] = snps_only.apply(lambda x: mutation(x["original AA"], x["mutated AA"]), axis=1)


,Contig,Type,SNP_POS,Start,End,Ref,Alt,ORF,Annotation,original codon,original AA,mutated codon,mutated AA,Mutation
0,NODE_1_length_648012_cov_4.992320,SNP,423752,423112,426795,G,T,AILJCPKN_00377,Respiratory nitrate reductase 1 alpha chain,AGT,S,AGT,S,Silence
1,NODE_1_length_648012_cov_4.992320,SNP,485246,485109,485660,C,T,AILJCPKN_00430,IS1182 family transposase ISSep1,TTC,F,TTT,F,Silence
2,NODE_1_length_648012_cov_4.992320,SNP,485486,485109,485660,G,A,AILJCPKN_00430,IS1182 family transposase ISSep1,AGG,R,AAG,K,Missense
3,NODE_1_length_648012_cov_4.992320,SNP,485534,485109,485660,T,C,AILJCPKN_00430,IS1182 family transposase ISSep1,TGT,C,CGT,R,Missense
4,NODE_1_length_648012_cov_4.992320,SNP,485569,485109,485660,A,G,AILJCPKN_00430,IS1182 family transposase ISSep1,CAT,H,CAG,Q,Missense
6,NODE_5_length_222064_cov_4.235925,SNP,83145,82211,83995,C,A,AILJCPKN_01835,UvrABC system protein C,CCT,P,CCA,P,Silence
7,NODE_6_length_167623_cov_3.900032,SNP,122497,122383,122976,T,C,AILJCPKN_02105,IS1182 family transposase ISSep1,TTT,F,CTT,L,Missense
8,NODE_6_length_167623_cov_3.900032,SNP,122553,122383,122976,C,T,AILJCPKN_02105,IS1182 family transposase ISSep1,GAC,D,GAT,D,Silence
9,NODE_6_length_167623_cov_3.900032,SNP,122593,122383,122976,A,G,AILJCPKN_02105,IS1182 family transposase ISSep1,AAT,N,AAG,K,Missense
10,NODE_6_length_167623_cov_3.900032,SNP,122641,122383,122976,G,A,AILJCPKN_02105,IS1182 family transposase ISSep1,GAT,D,GAA,E,Missense


In [87]:
snps_only["Mutation"].value_counts()

Mutation
Missense    18
Silence     11
Name: count, dtype: int64

In [88]:
#ALTERAR PARA SALVAR ARQUIVOS DIFERENTES
snps_only.to_excel("perc3_snps.xlsx")

In [89]:
missense = snps_only[snps_only["Mutation"] == "Missense"]
missense

,Contig,Type,SNP_POS,Start,End,Ref,Alt,ORF,Annotation,original codon,original AA,mutated codon,mutated AA,Mutation
2,NODE_1_length_648012_cov_4.992320,SNP,485486,485109,485660,G,A,AILJCPKN_00430,IS1182 family transposase ISSep1,AGG,R,AAG,K,Missense
3,NODE_1_length_648012_cov_4.992320,SNP,485534,485109,485660,T,C,AILJCPKN_00430,IS1182 family transposase ISSep1,TGT,C,CGT,R,Missense
4,NODE_1_length_648012_cov_4.992320,SNP,485569,485109,485660,A,G,AILJCPKN_00430,IS1182 family transposase ISSep1,CAT,H,CAG,Q,Missense
7,NODE_6_length_167623_cov_3.900032,SNP,122497,122383,122976,T,C,AILJCPKN_02105,IS1182 family transposase ISSep1,TTT,F,CTT,L,Missense
9,NODE_6_length_167623_cov_3.900032,SNP,122593,122383,122976,A,G,AILJCPKN_02105,IS1182 family transposase ISSep1,AAT,N,AAG,K,Missense
10,NODE_6_length_167623_cov_3.900032,SNP,122641,122383,122976,G,A,AILJCPKN_02105,IS1182 family transposase ISSep1,GAT,D,GAA,E,Missense
14,NODE_11_length_77465_cov_4.236280,SNP,11488,11324,12589,C,A,AILJCPKN_02580,UDP-N-acetylglucosamine 1-carboxyvinyltransfer...,GAC,D,GAA,E,Missense
15,NODE_15_length_14119_cov_5.840157,SNP,2673,1752,5177,A,G,AILJCPKN_02761,hypothetical protein,AGT,S,AGG,R,Missense
16,NODE_15_length_14119_cov_5.840157,SNP,2674,1752,5177,G,A,AILJCPKN_02761,hypothetical protein,AGT,S,AGA,R,Missense
17,NODE_15_length_14119_cov_5.840157,SNP,2802,1752,5177,C,T,AILJCPKN_02761,hypothetical protein,CCC,P,TCC,S,Missense


In [90]:
#ALTERAR PARA SALVAR ARQUIVOS DIFERENTES
missense.to_excel("perc3_Complete_Missense_mutations_data.xlsx")

In [91]:
SNP_summary_table = missense.groupby(["ORF", "Annotation"]).size().reset_index(name="Occurrences")
SNP_summary_table

,ORF,Annotation,Occurrences
0,AILJCPKN_00430,IS1182 family transposase ISSep1,3
1,AILJCPKN_02105,IS1182 family transposase ISSep1,3
2,AILJCPKN_02580,UDP-N-acetylglucosamine 1-carboxyvinyltransfer...,1
3,AILJCPKN_02761,hypothetical protein,3
4,AILJCPKN_02798,hypothetical protein,3
5,AILJCPKN_02811,Quinol oxidase subunit 3,1
6,AILJCPKN_02813,putative quinol oxidase subunit 2,4


In [92]:
#ALTERAR PARA SALVAR ARQUIVOS DIFERENTES
SNP_summary_table.to_excel("perc3_SNP_Missense_mutations_summary_table.xlsx")

In [93]:
#TESTAR INSERTIONS

insertions = results_df[(results_df["Type"] == "INSERTION") | (results_df["Type"] == "DELETION")]

def find_frameshift(ref, alt):
    insertion_size = len(alt) - len(ref)
    if insertion_size%3 != 0:
        return "FRAMESHIFT"
    if insertion_size%3 == 0:
        number_of_codons = insertion_size/3
        return f"INSERTION OF {number_of_codons} CODONS" 
insertions["FrameShift"] = insertions.apply(lambda x: find_frameshift(x["Ref"],x["Alt"]), axis=1)
insertions

/var/folders/hb/7z22q_f92bb8stzzy39dw7_80000gn/T/ipykernel_6679/2301349205.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insertions["FrameShift"] = insertions.apply(lambda x: find_frameshift(x["Ref"],x["Alt"]), axis=1)


,Contig,Type,SNP_POS,Start,End,Ref,Alt,ORF,Annotation,FrameShift
5,NODE_4_length_369082_cov_5.973587,DELETION,341858,341444,343249,GAAAAAAAA,GAAAAAAA,AILJCPKN_01733,UDP-N-acetyl-alpha-D-glucosamine C6 dehydratase,FRAMESHIFT


In [94]:
#ALTERAR PARA SALVAR ARQUIVOS DIFERENTES
insertions.to_excel("perc3_Insertion_deletion_PECqlqcoisa.xlsx")

In [100]:
from Bio import SeqIO

fasta_file = "ancestral.faa"
contig_id = "AILJCPKN_00764"
output_file = "hp_AILJCPKN_00764.faa"

# Busca o contig e salva em um novo arquivo
for record in SeqIO.parse(fasta_file, "fasta"):
    if record.id == contig_id:
        SeqIO.write(record, output_file, "fasta")
        print(f"Contig {contig_id} salvo em {output_file}")
        break



Contig AILJCPKN_00764 salvo em hp_AILJCPKN_00764.faa
